In [ ]:
#|export
from typing import List, Dict, Any
from titrations.basics import *

In [ ]:
#|export

class DosingLadder:
    ladder : Dict[str, List[Medication]]

    def __init__(self, ladder_dict : Dict[str, List[Medication]], single_class : bool = True) -> None:
        # TODO: ensure 'subladders' have the same number of steps

        # TODO: ensure each subladder consists of the same ingredient

        if single_class:
            pass # TODO: ensure all medications are of the same class

        self.ladder = ladder_dict
        # self.ingredient = None
        self.med_class = None

    @property
    def ingredients(self) -> List[Ingredient]:
        # TODO this should be okay if the checks in `__init__` are implemented
        return [self.ladder[med_name][0].ingredient for med_name in self.ladder]

    def get_subladder(self, medication : Ingredient | Medication):
        return self.ladder[medication.name]

    def _get_current_step_index(self, current_med : Medication):
        subladder = self.get_subladder(current_med)
        index = next((i for i, med in enumerate(subladder) if med.dose == current_med.dose), None)
        return index
    
    def _is_at_lowest_step(self, current_med : Medication):
        return current_med.dose == self.get_lowest_step(current_med).dose

    def _is_at_highest_step(self, current_med : Medication):
        return current_med.dose == self.get_highest_step(current_med).dose
    
    def get_next_step_up(self, current_med : Medication):
        current_dose_index = self._get_current_step_index(current_med)
        return self.get_subladder(current_med)[current_dose_index + 1]

    def get_next_step_down(self, current_med : Medication):
        current_dose_index = self._get_current_step_index(current_med)
        return self.get_subladder(current_med)[current_dose_index - 1]
    
    def get_lowest_step(self, medication: Ingredient | Medication):
        return self.get_subladder(medication)[0]

    def get_highest_step(self, medication: Ingredient | Medication):
        return self.get_subladder(medication)[-1]
    
    def get_current_medication_for_patient(self, patient : Patient):
        filtered = list(filter(lambda med: med.ingredient in self.ingredients, patient.medications))
        assert len(filtered) <= 1
        return filtered[0] if filtered else None
    
    @property
    def lowest_doses(self):
        return { med_name : self.ladder[med_name][0] for med_name in self.ladder}
    
    @property
    def highest_doses(self):
        return { med_name : self.ladder[med_name][-1] for med_name in self.ladder}

In [ ]:
metoprolol_succinate = Ingredient("metoprolol succinate")
carvedilol = Ingredient("carvedilol")
bisoprolol = Ingredient("bisoprolol")

In [ ]:
beta_blocker_ladder = DosingLadder({
    metoprolol_succinate.name: [
        Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
    ],
    carvedilol.name: [
        Medication(carvedilol, "3.125 mg", "PO", "BID"),
        Medication(carvedilol, "6.25 mg", "PO", "BID"),
        Medication(carvedilol, "12.5 mg", "PO", "BID"),
        Medication(carvedilol, "25 mg", "PO", "BID"),
    ],
    bisoprolol.name: [
        Medication(bisoprolol, "1.25 mg", "PO", "BID"),
        Medication(bisoprolol, "2.5 mg", "PO", "BID"),
        Medication(bisoprolol, "5 mg", "PO", "BID"),
        Medication(bisoprolol, "10 mg", "PO", "BID"),
    ]
})

In [ ]:
beta_blocker_ladder.get_subladder(metoprolol_succinate)

In [ ]:
beta_blocker_ladder.get_highest_step(metoprolol_succinate)

In [ ]:
beta_blocker_ladder.get_lowest_step(metoprolol_succinate)

In [ ]:
beta_blocker_ladder._get_current_step_index(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
    )

In [ ]:
beta_blocker_ladder._is_at_lowest_step(
    Medication(metoprolol_succinate, "12.5 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder._is_at_highest_step(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder.get_next_step_up(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder.get_next_step_down(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder.lowest_doses

In [ ]:
beta_blocker_ladder.highest_doses

In [ ]:
p = Patient(medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")])
beta_blocker_ladder.get_current_medication_for_patient(p)

In [84]:
#|export
import operator

class Rule:
    patient : Patient

    parameter : str
    operation : str
    threshold : Any

    operators = {
        "gt": operator.gt,
        "gte": operator.ge,
        "lt": operator.lt,
        "lte": operator.le,
        "eq": operator.eq,
        "neq": operator.ne,
        "in": operator.contains,
    }

    def __init__(self, parameter:str, operation:str, threshold:Any) -> None:
        assert parameter in VALID_PARAMETERS, "Not a valid parameter"
        assert operation in self.operators, f"Invalid operator {operation}"

        self.parameter = parameter
        self.operation = operation
        self.threshold = threshold

    def _is_satisfied(self, patient : Patient):
        patient_value = getattr(patient, self.parameter, None)
        if not patient_value: return False

        if self.operation == "in": return self.operators[self.operation](self.threshold, patient_value)
        else: return self.operators[self.operation](patient_value, self.threshold)

    def _get_eval_result_object(self, is_satisfied : bool) -> Any:
        return type('RuleEvalResult', (), {
            'is_satisfied': is_satisfied,
        })

    def evaluate(self, patient : Patient):
        is_satisfied = self._is_satisfied(patient)
        return self._get_eval_result_object(is_satisfied)
    
    def __repr__(self) -> str:
        return f"{self.parameter} {self.operation} {self.threshold}"

In [85]:
r = Rule('SBP', 'lt', 100)
eval_result = r.evaluate(Patient(SBP=100))
eval_result.is_satisfied

False

In [87]:
#|export
class ConditionalRule(Rule):
    condition : Rule

In [ ]:
#|export
class Titrator:
    patient : Patient
    dosing_ladder : DosingLadder

    current_medication : Medication
    
    titration_target = None

    rules = None
    initiation_rules = None
    titration_rules = None

    def __init__(self, patient : Patient,
                 dosing_ladder : DosingLadder) -> None:
        self.patient = patient
        self.dosing_ladder = dosing_ladder

        self.current_medication = self.dosing_ladder.get_current_medication_for_patient(patient)

    @property
    def current_ingredient(self) -> Ingredient:
        return self.current_medication.ingredient if self.current_medication else None

    @property
    def is_initiating(self) -> bool:
        return self.current_medication == None

    @property
    def is_titrating(self) -> bool:
        return not self.is_initiating


In [ ]:
t = Titrator(p, beta_blocker_ladder)

In [ ]:
t.current_ingredient

In [ ]:
t.is_initiating

In [ ]:
t.is_titrating